In [125]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import eikon as ek
#ek.set_app_key("898596c0c8884f6abd3faecd0d50522bf26c0242")

In [200]:
fullraw = pd.read_csv('fullraw.csv', index_col=1, parse_dates=True)
fullraw.index = pd.to_datetime(fullraw.index).strftime('%Y-%m-%d') # YYYY-MM-DD format

fullraw.sort_values(by=['TICKER', 'date'], inplace=True)
fullraw


/var/folders/7s/1b4ztzgn4fj7gc86hks6w5vc0000gn/T/ipykernel_82302/43796542.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  fullraw = pd.read_csv('fullraw.csv', index_col=1, parse_dates=True)


,PERMNO,SHRCD,TICKER,DISTCD,DIVAMT,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR
date,,,,,,,,,,,,
2015-07-13,87432,11,A,NaN,NaN,39.96,1579730.0,0.014213,39.95,39.96,1.0,1.0
2015-07-14,87432,11,A,NaN,NaN,40.49,2801301.0,0.013263,40.49,40.50,1.0,1.0
2015-07-15,87432,11,A,NaN,NaN,40.13,2247745.0,-0.008891,40.12,40.13,1.0,1.0
2015-07-16,87432,11,A,NaN,NaN,40.34,1826485.0,0.005233,40.34,40.35,1.0,1.0
2015-07-17,87432,11,A,NaN,NaN,39.95,2412373.0,-0.009668,39.94,39.95,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-07,91907,11,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,10.0,10.0
2020-01-08,91907,11,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,10.0,10.0
2020-01-09,91907,11,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,10.0,10.0


In [201]:
fullraw.isna().sum()

PERMNO           0
SHRCD            0
TICKER         403
DISTCD     1201815
DIVAMT     1201815
PRC            426
VOL            426
RET             23
BID            426
ASK            426
CFACPR          21
CFACSHR         21
dtype: int64

In [202]:
# remove all rows where TICKER is nan
fullraw = fullraw[fullraw['TICKER'].notna()]
fullraw

,PERMNO,SHRCD,TICKER,DISTCD,DIVAMT,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR
date,,,,,,,,,,,,
2015-07-13,87432,11,A,NaN,NaN,39.96000,1579730.0,0.014213,39.95000,39.96000,1.0,1.0
2015-07-14,87432,11,A,NaN,NaN,40.49000,2801301.0,0.013263,40.49000,40.50000,1.0,1.0
2015-07-15,87432,11,A,NaN,NaN,40.13000,2247745.0,-0.008891,40.12000,40.13000,1.0,1.0
2015-07-16,87432,11,A,NaN,NaN,40.34000,1826485.0,0.005233,40.34000,40.35000,1.0,1.0
2015-07-17,87432,11,A,NaN,NaN,39.95000,2412373.0,-0.009668,39.94000,39.95000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,13788,11,ZTS,NaN,NaN,164.70000,1023639.0,0.002557,164.62000,164.63000,1.0,1.0
2024-12-26,13788,11,ZTS,NaN,NaN,165.52000,2167214.0,0.004979,165.45000,165.46001,1.0,1.0
2024-12-27,13788,11,ZTS,NaN,NaN,164.60001,1800142.0,-0.005558,164.64000,164.64999,1.0,1.0


In [203]:
# -----------------------------------------
# 1 ▸ identify PERMNOs that ever show PRC < 0
# -----------------------------------------
neg_permnos = (
    fullraw.loc[fullraw['PRC'] < 0, 'PERMNO']
            .unique()           # numpy array of offending PERMNOs
)

print(f"{len(neg_permnos)} PERMNOs have at least one negative-flag price")
# optional: see the list
permno_list = sorted(neg_permnos.tolist())
print(permno_list[:200], "…")     # first 20 just to inspect

# -----------------------------------------
# 2 ▸ drop every row for those PERMNOs
# -----------------------------------------
fullraw = fullraw[~fullraw['PERMNO'].isin(neg_permnos)].copy()

print("cleaned rows:", len(fullraw))

6 PERMNOs have at least one negative-flag price
[64899, 76841, 82621, 84387, 89155, 90562] …
cleaned rows: 1202726


In [204]:
fullraw_clean.isna().sum()

PERMNO           0
SHRCD            0
TICKER           0
DISTCD     1187745
DIVAMT     1187745
PRC             20
VOL             20
RET             20
BID             20
ASK             20
CFACPR          19
CFACSHR         19
dtype: int64

In [221]:
mask_common = fullraw[fullraw['SHRCD'] // 10 == 1]          # first digit = 1

# mark *every* member of a duplicate group
dup_all = (
    mask_common
      .reset_index()
      .duplicated(subset=['date', 'TICKER'], keep=False)   # keep=False → mark every duplicate
)

dup_rows = (
    mask_common
      .reset_index()[dup_all]                  # pull them out
      .sort_values(['TICKER', 'date', 'PERMNO'])
)

dup_rows.to_csv('dup_rows.csv', index=False)

In [220]:

fullraw


,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH
date,,,,,,,,,,,
2015-07-13,10104,ORCL,11,40.80,10716991.0,0.010902,40.78000,40.79000,1.0,1.0,0.0
2015-07-14,10104,ORCL,11,40.78,9298777.0,-0.00049,40.75000,40.76000,1.0,1.0,0.0
2015-07-15,10104,ORCL,11,40.79,9326667.0,0.000245,40.80000,40.81000,1.0,1.0,0.0
2015-07-16,10104,ORCL,11,40.96,14861034.0,0.004168,40.96000,40.97000,1.0,1.0,0.0
2015-07-17,10104,ORCL,11,40.40,14574365.0,-0.013672,40.39000,40.40000,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,93436,TSLA,11,462.28,59351506.0,0.073572,462.25000,462.72000,1.0,1.0,0.0
2024-12-26,93436,TSLA,11,454.13,76392273.0,-0.01763,453.95001,454.01001,1.0,1.0,0.0
2024-12-27,93436,TSLA,11,431.66,82370345.0,-0.049479,431.56000,431.78000,1.0,1.0,0.0


In [212]:
# ensure date column is datetime

fullraw['date'] = pd.to_datetime(fullraw['date'])

# last trading date for every (ticker, permno)
last_seen = (
    fullraw.groupby(['TICKER', 'PERMNO'], as_index=False)
           .agg(last_date=('date', 'max'))
)

# pick the permno with the latest last_date for each ticker
recent_permno = (
    last_seen.sort_values(['TICKER', 'last_date'])  # newest last
            .groupby('TICKER')
            .tail(1)[['TICKER', 'PERMNO']]
)

# filter the main table
fullraw = fullraw.merge(recent_permno, on=['TICKER', 'PERMNO'], how='inner')

In [222]:
# number of unique tickers
print(len(fullraw['TICKER'].unique()))


566


In [215]:
import numpy as np
import pandas as pd

# ---------------------------------------------------
# 0 ▸ choose which distribution codes count as cash
# ---------------------------------------------------
def is_cash_div(row):
    dc = str(row['DISTCD'])
    return dc.startswith('12')         # all ordinary & special cash (1232,1272,…)

# ---------------------------------------------------
# 1 ▸ create a clean dividend column: value if cash, else 0
# ---------------------------------------------------
fullraw['DIV_CASH'] = np.where(fullraw.apply(is_cash_div, axis=1),
                               fullraw['DIVAMT'].fillna(0),
                               0.0)

# ---------------------------------------------------
# 2 ▸ aggregate so each (date, PERMNO) is unique
# ---------------------------------------------------
agg_map = {
    'TICKER'   : 'first',
    'SHRCD'    : 'first',
    'PRC'      : 'first',
    'VOL'      : 'first',
    'RET'      : 'first',
    'BID'      : 'first',
    'ASK'      : 'first',
    'CFACPR'   : 'first',
    'CFACSHR'  : 'first',
    'DIV_CASH' : 'sum',      # ← total cash distributed that day
}

fullraw = (
    fullraw
      .groupby(['date', 'PERMNO'], as_index=False)
      .agg(agg_map)
      .sort_values(['PERMNO', 'date'])
)

# now every (date, PERMNO) appears only once
print("any duplicates left?", fullraw.duplicated(['date', 'PERMNO']).any())

any duplicates left? False


In [223]:
allstocks = pd.read_csv('../../sp500_symbols.txt', header=None)
allstocks = allstocks[0].tolist()

# count number of tickers in allstocks
print(len(allstocks))

# count number of tickers in full_raw
print(len(fullraw['TICKER'].unique()))

print("Intersection: ", len(set(allstocks) & set(fullraw['TICKER'].unique())))

503
566
Intersection:  500


In [224]:
# keep intersection of fullraw and allstocks
fullraw = fullraw[fullraw['TICKER'].isin(allstocks)]
fullraw


,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH
date,,,,,,,,,,,
2015-07-13,10104,ORCL,11,40.80,10716991.0,0.010902,40.78000,40.79000,1.0,1.0,0.0
2015-07-14,10104,ORCL,11,40.78,9298777.0,-0.00049,40.75000,40.76000,1.0,1.0,0.0
2015-07-15,10104,ORCL,11,40.79,9326667.0,0.000245,40.80000,40.81000,1.0,1.0,0.0
2015-07-16,10104,ORCL,11,40.96,14861034.0,0.004168,40.96000,40.97000,1.0,1.0,0.0
2015-07-17,10104,ORCL,11,40.40,14574365.0,-0.013672,40.39000,40.40000,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,93436,TSLA,11,462.28,59351506.0,0.073572,462.25000,462.72000,1.0,1.0,0.0
2024-12-26,93436,TSLA,11,454.13,76392273.0,-0.01763,453.95001,454.01001,1.0,1.0,0.0
2024-12-27,93436,TSLA,11,431.66,82370345.0,-0.049479,431.56000,431.78000,1.0,1.0,0.0


In [225]:
fullraw.isna().sum()

PERMNO      0
TICKER      0
SHRCD       0
PRC         1
VOL         1
RET         1
BID         1
ASK         1
CFACPR      0
CFACSHR     0
DIV_CASH    0
dtype: int64

In [226]:
# show nan rows
fullraw[fullraw.isna().any(axis=1)]

,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH
date,,,,,,,,,,,
2016-07-21,90664,DXCM,11,NaN,NaN,None,NaN,NaN,4.0,4.0,0.0


In [227]:
fullraw['PRC_SPLIT_ADJ'] = fullraw['PRC'] / fullraw['CFACPR']
fullraw['VOL_ADJ'] = fullraw['VOL'] / fullraw['CFACSHR']
fullraw['BID_ADJ'] = fullraw['BID'] / fullraw['CFACPR']
fullraw['ASK_ADJ'] = fullraw['ASK'] / fullraw['CFACPR']
fullraw


/var/folders/7s/1b4ztzgn4fj7gc86hks6w5vc0000gn/T/ipykernel_82302/1173052541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullraw['PRC_SPLIT_ADJ'] = fullraw['PRC'] / fullraw['CFACPR']
/var/folders/7s/1b4ztzgn4fj7gc86hks6w5vc0000gn/T/ipykernel_82302/1173052541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullraw['VOL_ADJ'] = fullraw['VOL'] / fullraw['CFACSHR']
/var/folders/7s/1b4ztzgn4fj7gc86hks6w5vc0000gn/T/ipykernel_82302/1173052541.py:3: SettingWithCopyWarning: 
A value is trying to be set on

,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH,PRC_SPLIT_ADJ,VOL_ADJ,BID_ADJ,ASK_ADJ
date,,,,,,,,,,,,,,,
2015-07-13,10104,ORCL,11,40.80,10716991.0,0.010902,40.78000,40.79000,1.0,1.0,0.0,40.80,10716991.0,40.78000,40.79000
2015-07-14,10104,ORCL,11,40.78,9298777.0,-0.00049,40.75000,40.76000,1.0,1.0,0.0,40.78,9298777.0,40.75000,40.76000
2015-07-15,10104,ORCL,11,40.79,9326667.0,0.000245,40.80000,40.81000,1.0,1.0,0.0,40.79,9326667.0,40.80000,40.81000
2015-07-16,10104,ORCL,11,40.96,14861034.0,0.004168,40.96000,40.97000,1.0,1.0,0.0,40.96,14861034.0,40.96000,40.97000
2015-07-17,10104,ORCL,11,40.40,14574365.0,-0.013672,40.39000,40.40000,1.0,1.0,0.0,40.40,14574365.0,40.39000,40.40000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,93436,TSLA,11,462.28,59351506.0,0.073572,462.25000,462.72000,1.0,1.0,0.0,462.28,59351506.0,462.25000,462.72000
2024-12-26,93436,TSLA,11,454.13,76392273.0,-0.01763,453.95001,454.01001,1.0,1.0,0.0,454.13,76392273.0,453.95001,454.01001
2024-12-27,93436,TSLA,11,431.66,82370345.0,-0.049479,431.56000,431.78000,1.0,1.0,0.0,431.66,82370345.0,431.56000,431.78000


In [230]:
fullraw[fullraw['DIV_CASH'] > 0]

,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH,PRC_SPLIT_ADJ,VOL_ADJ,BID_ADJ,ASK_ADJ
date,,,,,,,,,,,,,,,
2015-10-09,10104,ORCL,11,38.10000,14656414.0,0.009501,38.10000,38.11000,1.0,1.0,0.15,38.10000,14656414.0,38.10000,38.11000
2016-01-04,10104,ORCL,11,35.75000,18784386.0,-0.017246,35.75000,35.76000,1.0,1.0,0.15,35.75000,18784386.0,35.75000,35.76000
2016-04-12,10104,ORCL,11,40.42000,9245042.0,0.004457,40.43000,40.44000,1.0,1.0,0.15,40.42000,9245042.0,40.43000,40.44000
2016-07-01,10104,ORCL,11,40.86000,11381612.0,0.001955,40.86000,40.87000,1.0,1.0,0.15,40.86000,11381612.0,40.86000,40.87000
2016-10-07,10104,ORCL,11,38.71000,11442032.0,-0.000257,38.71000,38.72000,1.0,1.0,0.15,38.71000,11442032.0,38.71000,38.72000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-29,93429,CBOE,11,179.50000,797800.0,-0.004644,179.50000,179.53999,1.0,1.0,0.55,179.50000,797800.0,179.50000,179.53999
2024-02-28,93429,CBOE,11,195.78000,529085.0,0.007544,195.67999,195.98000,1.0,1.0,0.55,195.78000,529085.0,195.67999,195.98000
2024-05-31,93429,CBOE,11,172.99001,1766239.0,-0.006754,173.05000,173.10001,1.0,1.0,0.55,172.99001,1766239.0,173.05000,173.10001


In [231]:
duplicates = fullraw.reset_index().duplicated(subset=['date', 'PERMNO'])
print(f"Number of duplicates per date and PERMNO: {duplicates.sum()}")
duplicates

Number of duplicates per date and PERMNO: 0


0          False
1          False
2          False
3          False
4          False
           ...  
1110739    False
1110740    False
1110741    False
1110742    False
1110743    False
Length: 1110744, dtype: bool

In [232]:
fullraw = fullraw.sort_values(['PERMNO', 'date'])
fullraw['DIV_CASH'] = fullraw['DIV_CASH'].fillna(0.0)      # keep 0 if no dividend

In [233]:
def add_adj_close(g):
    """
    g : DataFrame for one PERMNO, sorted by date ASC
    returns same frame with new ADJ_CLOSE column
    """
    # ➊ factor for each step = (price + dividend) / previous-price
    factor = (g['PRC_SPLIT_ADJ'] + g['DIV_CASH']) / g['PRC_SPLIT_ADJ'].shift()
    factor.iloc[0] = 1.0                                 # base period
    tr = factor.cumprod()                                # total-return index

    # ➋ rescale so the latest point equals the actual (split-adj) price
    scale = g['PRC_SPLIT_ADJ'].iloc[-1] / tr.iloc[-1]
    g['ADJ_CLOSE'] = tr * scale

    return g

fullraw = (
    fullraw
      .groupby('PERMNO', group_keys=False, sort=False)    # keeps big table order
      .apply(add_adj_close)
)

/var/folders/7s/1b4ztzgn4fj7gc86hks6w5vc0000gn/T/ipykernel_82302/2510193908.py:20: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(add_adj_close)


In [237]:
fullraw = fullraw.sort_values(['TICKER', 'date'])
fullraw

,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH,PRC_SPLIT_ADJ,VOL_ADJ,BID_ADJ,ASK_ADJ,ADJ_CLOSE
date,,,,,,,,,,,,,,,,
2015-07-13,87432,A,11,39.96000,1579730.0,0.014213,39.95000,39.96000,1.0,1.0,0.0,39.96000,1579730.0,39.95000,39.96000,36.994852
2015-07-14,87432,A,11,40.49000,2801301.0,0.013263,40.49000,40.50000,1.0,1.0,0.0,40.49000,2801301.0,40.49000,40.50000,37.485524
2015-07-15,87432,A,11,40.13000,2247745.0,-0.008891,40.12000,40.13000,1.0,1.0,0.0,40.13000,2247745.0,40.12000,40.13000,37.152237
2015-07-16,87432,A,11,40.34000,1826485.0,0.005233,40.34000,40.35000,1.0,1.0,0.0,40.34000,1826485.0,40.34000,40.35000,37.346655
2015-07-17,87432,A,11,39.95000,2412373.0,-0.009668,39.94000,39.95000,1.0,1.0,0.0,39.95000,2412373.0,39.94000,39.95000,36.985594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,13788,ZTS,11,164.70000,1023639.0,0.002557,164.62000,164.63000,1.0,1.0,0.0,164.70000,1023639.0,164.62000,164.63000,164.700000
2024-12-26,13788,ZTS,11,165.52000,2167214.0,0.004979,165.45000,165.46001,1.0,1.0,0.0,165.52000,2167214.0,165.45000,165.46001,165.520000
2024-12-27,13788,ZTS,11,164.60001,1800142.0,-0.005558,164.64000,164.64999,1.0,1.0,0.0,164.60001,1800142.0,164.64000,164.64999,164.600010


In [239]:
fullraw.to_csv('fullraw_processed.csv', index=True)

In [ ]:
fullraw.isna().sum()

PERMNO           0
TICKER           0
SHRCD            0
PRC              1
VOL              1
RET              1
BID              1
ASK              1
CFACPR           0
CFACSHR          0
DIV_CASH         0
PRC_SPLIT_ADJ    1
VOL_ADJ          1
BID_ADJ          1
ASK_ADJ          1
ADJ_CLOSE        2
dtype: int64

In [242]:
# Check for negative values in all columns except 'TICKER' and 'PERMNO'
cols_to_check = [col for col in fullraw.columns if col not in ['TICKER', 'PERMNO']]
neg_counts = {}
for col in cols_to_check:
    # Only check for negatives in numeric columns
    if pd.api.types.is_numeric_dtype(fullraw[col]):
        neg_counts[col] = (fullraw[col] < 0).sum()
    else:
        neg_counts[col] = None  # Not applicable for non-numeric columns
neg_counts



{'SHRCD': np.int64(0),
 'PRC': np.int64(0),
 'VOL': np.int64(0),
 'RET': None,
 'BID': np.int64(0),
 'ASK': np.int64(0),
 'CFACPR': np.int64(0),
 'CFACSHR': np.int64(0),
 'DIV_CASH': np.int64(0),
 'PRC_SPLIT_ADJ': np.int64(0),
 'VOL_ADJ': np.int64(0),
 'BID_ADJ': np.int64(0),
 'ASK_ADJ': np.int64(0),
 'ADJ_CLOSE': np.int64(0)}

In [243]:
fullraw

,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH,PRC_SPLIT_ADJ,VOL_ADJ,BID_ADJ,ASK_ADJ,ADJ_CLOSE
date,,,,,,,,,,,,,,,,
2015-07-13,87432,A,11,39.96000,1579730.0,0.014213,39.95000,39.96000,1.0,1.0,0.0,39.96000,1579730.0,39.95000,39.96000,36.994852
2015-07-14,87432,A,11,40.49000,2801301.0,0.013263,40.49000,40.50000,1.0,1.0,0.0,40.49000,2801301.0,40.49000,40.50000,37.485524
2015-07-15,87432,A,11,40.13000,2247745.0,-0.008891,40.12000,40.13000,1.0,1.0,0.0,40.13000,2247745.0,40.12000,40.13000,37.152237
2015-07-16,87432,A,11,40.34000,1826485.0,0.005233,40.34000,40.35000,1.0,1.0,0.0,40.34000,1826485.0,40.34000,40.35000,37.346655
2015-07-17,87432,A,11,39.95000,2412373.0,-0.009668,39.94000,39.95000,1.0,1.0,0.0,39.95000,2412373.0,39.94000,39.95000,36.985594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,13788,ZTS,11,164.70000,1023639.0,0.002557,164.62000,164.63000,1.0,1.0,0.0,164.70000,1023639.0,164.62000,164.63000,164.700000
2024-12-26,13788,ZTS,11,165.52000,2167214.0,0.004979,165.45000,165.46001,1.0,1.0,0.0,165.52000,2167214.0,165.45000,165.46001,165.520000
2024-12-27,13788,ZTS,11,164.60001,1800142.0,-0.005558,164.64000,164.64999,1.0,1.0,0.0,164.60001,1800142.0,164.64000,164.64999,164.600010


In [244]:
# Dollar spread in dollars per share
fullraw['SPREAD'] = fullraw['ASK_ADJ'] - fullraw['BID_ADJ']

# Optional: proportional spread (fraction of mid-price)
fullraw['REL_SPREAD'] = fullraw['SPREAD'] / ((fullraw['ASK_ADJ'] + fullraw['BID_ADJ']) / 2)

In [245]:
fullraw

,PERMNO,TICKER,SHRCD,PRC,VOL,RET,BID,ASK,CFACPR,CFACSHR,DIV_CASH,PRC_SPLIT_ADJ,VOL_ADJ,BID_ADJ,ASK_ADJ,ADJ_CLOSE,SPREAD,REL_SPREAD
date,,,,,,,,,,,,,,,,,,
2015-07-13,87432,A,11,39.96000,1579730.0,0.014213,39.95000,39.96000,1.0,1.0,0.0,39.96000,1579730.0,39.95000,39.96000,36.994852,0.01000,0.000250
2015-07-14,87432,A,11,40.49000,2801301.0,0.013263,40.49000,40.50000,1.0,1.0,0.0,40.49000,2801301.0,40.49000,40.50000,37.485524,0.01000,0.000247
2015-07-15,87432,A,11,40.13000,2247745.0,-0.008891,40.12000,40.13000,1.0,1.0,0.0,40.13000,2247745.0,40.12000,40.13000,37.152237,0.01000,0.000249
2015-07-16,87432,A,11,40.34000,1826485.0,0.005233,40.34000,40.35000,1.0,1.0,0.0,40.34000,1826485.0,40.34000,40.35000,37.346655,0.01000,0.000248
2015-07-17,87432,A,11,39.95000,2412373.0,-0.009668,39.94000,39.95000,1.0,1.0,0.0,39.95000,2412373.0,39.94000,39.95000,36.985594,0.01000,0.000250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,13788,ZTS,11,164.70000,1023639.0,0.002557,164.62000,164.63000,1.0,1.0,0.0,164.70000,1023639.0,164.62000,164.63000,164.700000,0.01000,0.000061
2024-12-26,13788,ZTS,11,165.52000,2167214.0,0.004979,165.45000,165.46001,1.0,1.0,0.0,165.52000,2167214.0,165.45000,165.46001,165.520000,0.01001,0.000060
2024-12-27,13788,ZTS,11,164.60001,1800142.0,-0.005558,164.64000,164.64999,1.0,1.0,0.0,164.60001,1800142.0,164.64000,164.64999,164.600010,0.00999,0.000061


In [247]:
fullraw.to_csv('fullraw_processed.csv', index=True)

In [248]:
fullraw.columns

Index(['PERMNO', 'TICKER', 'SHRCD', 'PRC', 'VOL', 'RET', 'BID', 'ASK',
       'CFACPR', 'CFACSHR', 'DIV_CASH', 'PRC_SPLIT_ADJ', 'VOL_ADJ', 'BID_ADJ',
       'ASK_ADJ', 'ADJ_CLOSE', 'SPREAD', 'REL_SPREAD'],
      dtype='object')

In [249]:
# --------------------------------------------
# helper — wide pivot: dates as index, tickers as columns
# --------------------------------------------
def pivot_wide(df, value_col):
    """Return date-indexed table of <value_col> with TICKER columns."""
    wide = (
        df.reset_index()                       # date was index
          .pivot(index='date', columns='TICKER', values=value_col)
          .sort_index()                        # ascending calendar
    )
    return wide

# --------------------------------------------
# 1 ▸ Prices  (split + dividend adjusted close)
# --------------------------------------------
prices = pivot_wide(fullraw, 'ADJ_CLOSE')
prices.to_csv('prices.csv')

# 2 ▸ Returns
rets = pivot_wide(fullraw, 'RET')
rets.to_csv('ret.csv')

# 3 ▸ Relative bid-ask spread  (percentage)
spreads = pivot_wide(fullraw, 'REL_SPREAD')
spreads.to_csv('spreads.csv')

# 4 ▸ Dollar bid-ask spread
dollar_spread = pivot_wide(fullraw, 'SPREAD')
dollar_spread.to_csv('dollar_spread.csv')

# 5 ▸ Split-adjusted volumes (shares)
volumes = pivot_wide(fullraw, 'VOL_ADJ')
volumes.to_csv('volumes_shares.csv')

print("✔  files written:")
for fn in ['prices.csv','ret.csv','spreads.csv','dollar_spread.csv','volumes_shares.csv']:
    print("   ", fn, " – ", os.path.getsize(fn)//1024, "KB")

✔  files written:


NameError: name 'os' is not defined

# Short fee data

In [4]:
import pandas as pd
from pathlib import Path
import os

def combine_short_fee_files(folder_path: str, output_filename: str = 'short_fees.csv'):
    """
    Combines multiple short fee CSV files from a folder into a single file.

    Each input CSV is expected to have 'time' and 'fee' columns.
    The function will create a single CSV with dates as the index and tickers
    (derived from filenames) as columns, with the 'fee' data as values.

    Parameters
    ----------
    folder_path : str
        The path to the folder containing the CSV files.
    output_filename : str, optional
        The name of the output CSV file, by default 'short_fees.csv'.
    """
    # Use pathlib for modern and robust path handling
    p = Path(folder_path)
    
    if not p.is_dir():
        print(f"Error: Folder not found at '{folder_path}'")
        return

    all_stock_data = []
    
    # Efficiently get a list of all csv files
    csv_files = list(p.glob('*.csv'))
    
    if not csv_files:
        print(f"Warning: No CSV files were found in '{folder_path}'.")
        return

    print(f"Found {len(csv_files)} CSV files. Processing...")

    for file_path in csv_files:
        try:
            # 1. Extract the ticker from the filename (e.g., 'AAPL.csv' -> 'AAPL')
            ticker = file_path.stem

            # 2. --- CORRECTED --- Read only the 'time' and 'fee' columns
            df = pd.read_csv(file_path, usecols=['time', 'fee'])

            # 3. Set 'time' as the index
            df['time'] = pd.to_datetime(df['time'])
            df.set_index('time', inplace=True)

            # 4. Handle duplicate timestamps within a single file to prevent errors
            if df.index.has_duplicates:
                df = df[~df.index.duplicated(keep='last')]

            # 5. --- CORRECTED --- Rename the 'fee' column to the ticker name
            df.rename(columns={'fee': ticker}, inplace=True)
            
            # 6. Add the processed single-stock DataFrame to our list
            all_stock_data.append(df)

        except Exception as e:
            print(f"Could not process file {file_path.name}: {e}")
            
    if not all_stock_data:
        print("No files were successfully processed.")
        return

    # 7. Concatenate all DataFrames in the list into one.
    print("Combining all data...")
    final_df = pd.concat(all_stock_data, axis=1)

    # Optional: Sort columns alphabetically for consistency
    final_df = final_df.sort_index(axis=1)

    # 8. Save the final combined DataFrame to a new CSV file
    final_df.to_csv(output_filename)
    
    print(f"\n✅ Success! Created '{output_filename}' with shape {final_df.shape}.")
    print(f"   Index: Date, Columns: Tickers, Values: Fee")



In [13]:

# --- How to use the function ---

# 1. Define the path to your folder with the 500 CSVs
#    (Replace with your actual folder path)
#    Example for Mac/Linux:
#    data_folder = '/Users/your_name/Documents/Short_Fee_Data'
#    Example for Windows:
#    data_folder = 'C:\\Users\\your_name\\Documents\\Short_Fee_Data'

data_folder = '../../../../data/ibkr_short_fee/'

# 2. Call the function
combine_short_fee_files(data_folder)

Found 503 CSV files. Processing...
Combining all data...

✅ Success! Created 'short_fees.csv' with shape (2551, 503).
   Index: Date, Columns: Tickers, Values: Fee


In [14]:
shortfee = pd.read_csv('short_fees.csv', index_col=0, parse_dates=True)
prices = pd.read_csv('prices.csv', index_col=0, parse_dates=True)

# intersection of tickers in shortfee and prices
common_tickers = set(shortfee.columns) & set(prices.columns)
print(f"Number of common tickers: {len(common_tickers)}")


Number of common tickers: 499


In [15]:
diff = set(prices.columns) - set(shortfee.columns)
diff

{'SMCI'}